**Welcome to PyLabRobot!**

PyLabRobot is a universal Python interface to liquid-handling robots. Liquid-handling robots aspirate and dispense precise volumes
of liquid in a Cartesian coordinate system, essentially the same as hand pipetting.

PLR defines a universal interface class called LiquidHandler that provides generic methods for controlling robots
such as aspirate and dispense. This class can be instantiated with one of several backends (or drivers)
that convert generic commands to machine-specific commands. This makes it easy to write code that will
be translatable across many different machines.

First we will import `LiquidHandler`, a backend called `ChatterBoxBackend` that prints the text
output of our commands, a class `Visualizer` that provides a visualization of the robot deck as we
run commands, and a class `STARLetDeck` that will represent the deck of a Hamilton Microlab STARLet, one of
the most widely used liquid handling robots.

In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck


In [3]:
lh = LiquidHandler(backend=ChatterBoxBackend(), deck=OTDeck())
await lh.setup()

Setting up the robot.
Resource deck was assigned to the robot.
Resource trash_container was assigned to the robot.


In [3]:
vis = Visualizer(resource=lh)
await vis.setup()

Websocket server started at http://127.0.0.1:2122
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


Now we will import a tip carrier, a plate carrier, a plate, and a tip rack.

In [5]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L
)


In [6]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = tip_rack1 = HTF_L(name='tips_01', with_tips=False)
tip_car[1] = tip_rack2 = HTF_L(name='tips_02', with_tips=False)
tip_car[2] = tip_rack3 = HTF_L(name='tips_03', with_tips=False)
tip_car[3] = tip_rack4 = HTF_L(name='tips_04', with_tips=False)
tip_car[4] = tip_rack5 = HTF_L(name='tips_05', with_tips=False)
lh.deck.assign_child_resource(tip_car, rails=15)


Resource tip carrier was assigned to the robot.


In [7]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = plate_1 = Cos_96_DW_1mL(name='plate_01')
plt_car[1] = plate_2 = Cos_96_DW_1mL(name='plate_02')
plt_car[2] = plate_3 = Cos_96_DW_1mL(name='plate_03')
lh.deck.assign_child_resource(plt_car, rails=8)


Resource plate carrier was assigned to the robot.


In [8]:
tip_rack1.fill()


In [ ]:
tip_rack4 = lh.deck.get_resource("tips_04")
tip_rack4.set_tip_state([[True]*6 + [False]*6]*8)
tip_rack3.set_tip_state([[True, False]*6]*8)
tip_rack2.set_tip_state([[True, True, False, False]*3]*8)


In [ ]:
plate_1_liquids = [[(None, 500)]]*96
plate_1.set_well_liquids(plate_1_liquids)
plate_2_liquids = [[(None, 100)], [(None, 500)]]*(96//2)
plate_2.set_well_liquids(plate_2_liquids)


In [ ]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking
set_tip_tracking(True), set_volume_tracking(True)


In [ ]:
await lh.pick_up_tips(tip_rack1["A1", "B2", "C3", "D4"])
await lh.drop_tips(tip_rack1["A1", "B2", "C3", "D4"])


In [ ]:
await lh.pick_up_tips(tip_rack1["A1"])


In [ ]:
await lh.aspirate(plate_1["A1"], vols=[300])


In [ ]:
await lh.dispense(plate_1["A1"], vols=[300])


In [ ]:
await lh.return_tips()


In [ ]:
await lh.pick_up_tips96(tip_rack1)


In [ ]:
await lh.aspirate96(plate_1, volume=100)


In [ ]:
await lh.dispense96(plate_3, volume=100)


In [ ]:
await lh.drop_tips96(tip_rack1)


In [ ]:
await vis.stop()
